# Reinforced learning, työllistyminen ja Suomen sosiaaliturva

Tässä tehdään laskelmat artikkelia varten. Käytössä on gym-ympäristö _unemployment-v1_ , johon on toteutettu yksityiskohtaisempi työttömyysturvamalli.

In [1]:
# for Colab, install fin_benefits and unemployment-gym from Github
#!pip install -q git+https://github.com/ajtanskanen/benefits.git  
#!pip install -q git+https://github.com/ajtanskanen/econogym.git
#!pip install -q git+https://github.com/ajtanskanen/lifecycle-rl.git

# and then restart kernel
  
  # For a specific version:
#!pip install tensorflow==1.15
#!pip install stable-baselines==2.8
  
# restart kernel after running pip's

Then load all modules.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lifecycle_rl import Lifecycle

%matplotlib inline
%pylab inline

# varoitukset piiloon (Stable baseline ei ole vielä Tensorflow 2.0-yhteensopiva, ja Tensorflow 1.5 valittaa paljon)
import warnings
warnings.filterwarnings('ignore')

pop_size=1_000

ModuleNotFoundError: No module named 'baselines.common.vec_env.vec_monitor'

# Nykymalli 

Lasketaan työllisyysasteet nykymallissa.

In [ ]:
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=True,perustulo=False,randomness=True,deterministic=False)
cc1.explain()
cc1.train(steps=10_000_000,cont=True,rlmodel='acktr',save='perusmalli_0',batch=1,debug=True,bestname='best_perus')

In [ ]:
cc1.simulate(pop=10_000,deterministic=False,load='tmp/best_perus',rlmodel='acktr',save='perusmalli')

# EK:n malli

Lasketaan vertailukelpoiset työllisyysasteet EK:n ehdottamalla mallilla. Mallissa on toteuttu muutoksia ansiosidonnaiseen työttömyysturvaan, asumistukeen, toimeentulotukeen ja verotukseen.

In [ ]:
cc2=Lifecycle(env='unemploymentEK-v1',minimal=False,mortality=True,perustulo=False,randomness=True)
cc2.train(steps=10_000_000,cont=False,rlmodel='acktr',save='ekn_malli',batch=1,start_from='tmp/best_perus',bestname='best_ek')
cc2.simulate(pop=10_000,deterministic=False,load='tmp/best_ek',rlmodel='acktr',save='eknmalli')

In [ ]:
cc2.compare_with(cc1)

# Verifiointi

Ajetaan sama simulaatio kaksi kertaa ja tarkastetaan että tulokset ovat sama. Tässä ideana on varmistaa, että satunnaisuus ei sotke tulosten arviointia.

In [ ]:
min_pop=1_00
cc1=Lifecycle(env='unemployment-v1',minimal=False,mortality=False,perustulo=False,randomness=False)
cc1.simulate(pop=min_pop,rlmodel='acktr',load='perusmalli_0',plot=False,debug=True)
cc10=Lifecycle(env='unemployment-v1',minimal=False,mortality=False,perustulo=False,randomness=False)
cc10.simulate(pop=min_pop,rlmodel='acktr',load='perusmalli_0',plot=False,debug=True)
cc10.compare_with(cc1)

In [ ]:
cc10=Lifecycle(env='unemployment-v1',minimal=False,mortality=False,perustulo=False,randomness=False)
cc10.simulate(pop=2000,rlmodel='acktr',load='perusmalli_0',plot=True,debug=False,deterministic=False)
cc10.compare_with(cc1)

In [ ]:
cc0=Lifecycle(env='unemployment-v1',minimal=False,mortality=False,perustulo=False,randomness=False)
cc0.distribution(n=50,steps1=1_000_000,steps2=1_000_000,pop=1_000,debug=False)

# 300 vs 400 päivän kesto ansiosidonnaisessa

Mikä on alle kolmen vuoden työhistorian lyhyemmän (300 pv) ansiosidonnaisen päivärahan enimmäiskeston vaikutus työllisyyteen? Kokeillaan miten työllisyyteen vaikuttaisi sen pidentäminen 400 päivään.

In [ ]:
cc1_400=Lifecycle(env='unemployment-v1',minimal=False,ansiopvraha_kesto300=400,randomness=False)
cc1_400.train(steps=2_000_000,cont=False,rlmodel='acktr',save='300_vs_400',start_from='perusmalli_0')
cc1_400.simulate(pop=pop_size)
cc1_400.compare_with(cc1)

# Työssäoloehdon pituus 12 kk

Entä jos työssäoloehto olisikin 12 kuukautta pitkä nykyisen 6 kuukauden sijaan?

In [ ]:
cc1_toe=Lifecycle(env='unemployment-v1',minimal=False,ansiopvraha_toe=1.0,randomness=False)
cc1_toe.train(steps=2_000_000,cont=False,rlmodel='acktr',save='toe_12',batch=1,start_from='perusmalli_0')
cc1_toe.simulate(pop=pop_size)

In [ ]:
cc_vert=Lifecycle(env='unemployment-v1',minimal=False)
cc_vert.load_sim('perusmalli_0')
cc1_toe.compare_with(cc_vert)

# Ansiosidonnaisen päivärahan lyhennys 50 pv

Tarkastellaan, miten työllisyyteen vaikuttaisi ansiosidonnaisen päivärahan lyhentäminen 50 päivällä. Tällöin alle kolmen vuoden työhistorialla ansiosidonnaisen päivärahan enimmäiskesto olisi 250 pv ja pidemmällä työhistorialla enimmäiskesto olisi 350 pv.

In [ ]:
cc1_350=Lifecycle(env='unemployment-v1',minimal=False,ansiopvraha_kesto300=250,ansiopvraha_kesto400=350,randomness=False)
cc1_350.train(train=True,steps=2_000_000,cont=False,rlmodel='acktr',save='50_short',start_from='perusmalli_0')
cc1_350.simulate(pop=pop_size)

In [ ]:
cc1_350.compare_with(cc1)

# Perustulo

In [ ]:
cc1_350=Lifecycle(env='unemployment-v1',minimal=False,perustulo=True,randomness=False)
cc1_350.train(train=True,steps=2_000_000,cont=False,rlmodel='acktr',save='perustulo',start_from='perusmalli_0')
cc1_350.simulate(pop=pop_size)

In [ ]:
cc2.compare_with(cc1)

# Jakaumat

Testataan sitten mallin konvergenssia.

In [ ]:
cc=Lifecycle(env='unemployment-v1',timestep=0.25,randomness=True,mortality=False)
cc.run_results(debug=False,steps1=50_000_000,steps2=1_000_000,n=30,pop=10000,deterministic=False,train=False,predict=False)

In [ ]:
cc=Lifecycle(env='unemployment-v1',timestep=0.25,randomness=True,mortality=False,deterministic=True)
cc.run_results(debug=False,steps1=3_000_000,steps2=100_000,n=3,pop=1_000,deterministic=True,\
               train=True,predict=True,batch=1,results='res100_',results_dir='results',save_dir='saved')

In [ ]:
for k in range(-1,14):
    print(cc.env.state_decode(cc.env.state_encode_nomort(k,0,200,3000,40,50,6000,1,2,50,7000)))